# 05 - Pix2Pix Inference

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

# from botocore.exceptions import ClientError
# import boto3
# import calendar
# import io
# from itertools import product
# import numpy as np
import os
import pandas as pd
# import random
# import tifffile as tif
# from tqdm import tqdm

import torch
import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms

# from models.pix2pix_model import Pix2PixModel

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

/Users/loaner/.pyenv/versions/3.8.15/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create train, validation, test splits

## Download train data to local

In [20]:
%cd /Users/loaner/projects/dd-biomassters/pix2pix
from data.biomassters_dataset import BioMasstersDataset
from torch.utils.data import DataLoader
from tqdm import tqdm

class Options():
    def __init__(self, phase='train', max_dataset_size=100): 
        self.dataroot = '../data'
        self.phase = phase
        self.max_dataset_size = max_dataset_size

/Users/loaner/projects/dd-biomassters/pix2pix


In [21]:
opt = Options()
bm = BioMasstersDataset(opt)
dataloader = DataLoader(bm, batch_size=1)

In [25]:
for i, data in enumerate(dataloader):
    print(i, data['chip_id'], data['A'].shape, data['B'].shape)

0 ['1a1f591d'] torch.Size([1, 180, 256, 256]) torch.Size([1, 1, 256, 256])
1 ['5521a4ea'] torch.Size([1, 180, 256, 256]) torch.Size([1, 1, 256, 256])
2 ['b910d1a7'] torch.Size([1, 180, 256, 256]) torch.Size([1, 1, 256, 256])
3 ['eee3ce3a'] torch.Size([1, 180, 256, 256]) torch.Size([1, 1, 256, 256])
4 ['49abd286'] torch.Size([1, 180, 256, 256]) torch.Size([1, 1, 256, 256])
5 ['187cf18b'] torch.Size([1, 180, 256, 256]) torch.Size([1, 1, 256, 256])
6 ['243bad58'] torch.Size([1, 180, 256, 256]) torch.Size([1, 1, 256, 256])
7 ['8baffdbd'] torch.Size([1, 180, 256, 256]) torch.Size([1, 1, 256, 256])
8 ['138bc232'] torch.Size([1, 180, 256, 256]) torch.Size([1, 1, 256, 256])
9 ['2003e1cb'] torch.Size([1, 180, 256, 256]) torch.Size([1, 1, 256, 256])
10 ['10cddd46'] torch.Size([1, 180, 256, 256]) torch.Size([1, 1, 256, 256])
11 ['31c77f10'] torch.Size([1, 180, 256, 256]) torch.Size([1, 1, 256, 256])
12 ['5f5e708d'] torch.Size([1, 180, 256, 256]) torch.Size([1, 1, 256, 256])
13 ['9d73fff0'] torch.

In [26]:
%cd /Users/loaner/projects/dd-biomassters

/Users/loaner/projects/dd-biomassters


## Test via command line interface

In [29]:
!python ./pix2pix/test.py --dataroot ./data --name biomassters --model pix2pix --direction AtoB --input_nc 180 \
    --dataset_mode biomassters --gpu_ids -1 --results_dir ./results/ --phase validation \
    --max_dataset_size 10 --preprocess "" --no_dropout


----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./data                        	[default: data]
             dataset_mode: biomassters                   	[default: aligned]
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: -1                            	[default: 0]
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 180                           	[default: 3]
                  isTrain: False                         	[default: None]
           

## Train dev

In [5]:
!python ./pix2pix/train.py --dataroot ./data --name biomassters --model pix2pix --direction AtoB \
    --input_nc 180 --output_nc 1 --dataset_mode biomassters --gpu_ids -1 --phase train \
    --max_dataset_size 5 --preprocess "" \
    --n_epochs 10 --n_epochs_decay 10 \
    --display_id 1 --display_freq 5 --print_freq 5
# --num_threads 1 \

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: ./data                        	[default: data]
             dataset_mode: biomassters                   	[default: aligned]
                direction: AtoB                          
              display_env: main                          
             display_freq: 5                             	[default: 400]
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest     

In [42]:
!python ./pix2pix/test.py --dataroot ./data --name biomassters --model pix2pix --direction AtoB \
    --input_nc 180 --output_nc 1 --dataset_mode biomassters --gpu_ids -1 --phase train \
    --max_dataset_size 5 --preprocess "" \
    --no_dropout --results_dir ./results/ 

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./data                        	[default: data]
             dataset_mode: biomassters                   	[default: aligned]
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: -1                            	[default: 0]
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 180                           	[default: 3]
                  isTrain: False                         	[default: None]
           